# Multitask learning notebook

In this notebook we'll do some toy experiments with a multitask learning model.

In [1]:
import torch.optim as opt
from torch.nn import CrossEntropyLoss
from mlearn.utils.metrics import Metrics
from mlearn.utils.train import train_mtl_model
from mlearn.data.dataset import GeneralDataset
from mlearn.data.clean import Preprocessors, Cleaner
from mlearn.data.loaders import hoover, davidson, garcia, semeval_sentiment
from mlearn.modeling.multitask import EmbeddingLSTMClassifier, OnehotLSTMClassifier, OnehotMLPClassifier

## Load the datasets to be used

In [2]:
cleaner = Cleaner(['lower', 'hashtag', 'url', 'user'])
data_path = '../tests/data/'

# Hoover dataset
mftc = hoover(cleaners = cleaner, data_path = data_path)
mftc.build_token_vocab(mftc.data)
mftc.build_label_vocab(mftc.data)

# Davidson dataset
off = davidson(cleaners = cleaner, data_path = data_path)
off.build_token_vocab(off.data)
off.build_label_vocab(off.data)

# Sentiment analysis dataset
sent = semeval_sentiment(cleaners = cleaner, data_path = data_path)
sent.build_token_vocab(sent.data)
sent.build_label_vocab(sent.data)

Loading Hoover et al. (train): 999it [00:04, 239.64it/s]
Encode label vocab: 100%|██████████| 11/11 [00:00<00:00, 13462.90it/s]
Loading Davidson et al. (train): 887it [00:04, 187.80it/s]
Encode label vocab: 100%|██████████| 3/3 [00:00<00:00, 2270.87it/s]
Loading Semeval (2016) (train): 999it [00:04, 218.00it/s]
Encode label vocab: 100%|██████████| 3/3 [00:00<00:00, 8224.13it/s]


## Create Batches of train and dev

## Initialize models

In [3]:
input_dims = [sent.vocab_size(), off.vocab_size(), mftc.vocab_size()]
output_dims = [sent.label_count(), off.label_count(), mftc.label_count()]
shared_dim, hidden_dims, embedding_dim = 256, [64, 128, 300], 128
dropout = 0.4
print(input_dims)

[3434, 2934, 2845]


In [4]:
ohmlp = OnehotMLPClassifier(input_dims = input_dims, shared_dim = shared_dim, hidden_dims = hidden_dims,
                            output_dims = output_dims, dropout = dropout)
optim = opt.SGD(ohmlp.parameters(), lr = 0.01)
loss = CrossEntropyLoss()
m = Metrics(['f1-score'], display_metric = 'f1-score', early_stop = 'f1-score')
train_mtl_model(ohmlp, training_datasets = [sent, off, mftc], save_path = '.', opt = optim, loss_func = loss,
                metrics = m, batch_size = 32, epochs = 2, clip = 1.0, patience = 3, dev = sent, dev_task_id = 0,
                batches_per_epoch = 200, shuffle_data = True, dataset_weights = [0.6, 0.2, 0.2],
                loss_weights = [0.6, 0.3, 0.1], onehot = True)

Batch:   0%|          | 0/200 [00:00<?, ?it/s]

OnehotMLPClassifier(
  (all_parameters): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 64x3434]
      (1): Parameter containing: [torch.FloatTensor of size 64]
      (2): Parameter containing: [torch.FloatTensor of size 64x2934]
      (3): Parameter containing: [torch.FloatTensor of size 64]
      (4): Parameter containing: [torch.FloatTensor of size 64x2845]
      (5): Parameter containing: [torch.FloatTensor of size 64]
      (6): Parameter containing: [torch.FloatTensor of size 128x64]
      (7): Parameter containing: [torch.FloatTensor of size 128]
      (8): Parameter containing: [torch.FloatTensor of size 300x128]
      (9): Parameter containing: [torch.FloatTensor of size 300]
      (10): Parameter containing: [torch.FloatTensor of size 3x300]
      (11): Parameter containing: [torch.FloatTensor of size 3]
      (12): Parameter containing: [torch.FloatTensor of size 3x300]
      (13): Parameter containing: [torch.FloatTensor of size 3]
      (14): Pa

/Users/zeerakw/Documents/PhD/projects/tools/mlearn/mlearn/utils/train.py:205: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(), clip)  # Prevent exploding gradients
/Users/zeerakw/.virtualenvs/mlearn/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)

Batch:  14%|█▍        | 29/200 [00:04<00:26,  6.34it/s, batch_loss=0.0198, diff=0, epoch_loss=0.0155, f1-score=0.288, task=0]


Batch:  30%|███       | 60/200 [00:10<00:30,  4.60it/s, batch_loss=0.0075, diff=0, epoch_loss=0.0153, f1-score=0.286, task=2]


Batch:  46%|████▌     | 91/200 [00:16<00:18,  5.83it/s, batch_loss=0.0075, diff=-.107, epoch_loss=0.0155, f1-score=0.302, task=2]


...

Batch:  84%|████████▎ | 167/200 [00:35<00:07,  4.18it/s, batch_loss=0.0075, diff=-.544, epoch_loss=0.0140, f1-score=0.344, task=2]


Batch:  98%|█████████▊| 197/200 [00:41<00:00,  4.55it/s, batch_loss=0.0091, diff=0.118, epoch_loss=0.0138, f1-score=0.348, task=1]


Training model: 100%|██████████| 2/2 [01:19<00:00, 39.86s/it, epoch_loss=0.0139]                                              


Exception occurred: Labels have already been processed. Dataset name: Semeval (2016)
Exception occurred: Labels have already been processed. Dataset name: Semeval (2016)
Exception occurred: Labels have already been processed. Dataset name: Semeval (2016)
Exception occurred: Labels have already been processed. Dataset name: Semeval (2016)
Exception occurred: Labels have already been processed. Dataset name: Semeval (2016)
Exception occurred: Labels have already been processed. Dataset name: Semeval (2016)
Exception occurred: Labels have already been processed. Dataset name: Semeval (2016)
Exception occurred: Labels have already been processed. Dataset name: Semeval (2016)
Exception occurred: Labels have already been processed. Dataset name: Semeval (2016)
Exception occurred: Labels have already been processed. Dataset name: Semeval (2016)
Exception occurred: Labels have already been processed. Dataset name: Semeval (2016)
Exception occurred: Labels have already been processed. Dataset n

([0.014845838099718093, 0.013874369596596808], [], None, [])